In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage.io as io


In [ ]:
def get_file(file_dir):
    ''' 得到图片以及图片对应的labels

    Returns:
        images: image directiories, list, string
        labels: label, list, int
    '''

    images = []
    temp = []
    for root, sub_folders, files in os.walk(file_dir):    #获得根目录，子文件夹和文件夹下所有的文件
        for name in files:
            images.append(os.path.join(root, name))       #获得图片的绝对路径
        for name in sub_folders:
            temp.append(os.path.join(root, name))        #获得子文件夹即标签的绝对路径


    labels = []
    for one_folder in temp:
        number_img = len(os.listdir(one_folder))
        letter = one_folder.split('/')[-1]             #分割路径，取得最后的label
        print(letter)
        if letter == 'A':
            labels = np.append(labels, number_img * [1])
        elif letter == 'B':
            labels = np.append(labels, number_img * [2])
        elif letter == 'C':
            labels = np.append(labels, number_img * [3])
        elif letter == 'D':
            labels = np.append(labels, number_img * [4])
        elif letter == 'E':
            labels = np.append(labels, number_img * [5])
        elif letter == 'F':
            labels = np.append(labels, number_img * [6])
        elif letter == 'G':
            labels = np.append(labels, number_img * [7])
        elif letter == 'H':
            labels = np.append(labels, number_img * [8])
        elif letter == 'I':
            labels = np.append(labels, number_img * [9])
        else:
            labels = np.append(labels, number_img * [10])

    # shuffle  组合image和label并且打乱顺序
    temp = np.array([images, labels])
    temp = temp.transpose()
    np.random.shuffle(temp)

    images_list = list(temp[:, 0])
    labels_list = list(temp[:, 1])
    labels_list = [int(float(i)) for i in labels_list]
    return images_list, labels_list



In [ ]:
file_dir = '../data/notMNIST_small'
images, labels = get_file(file_dir)
print("hello :",images)
print(labels)

In [ ]:
#TFRecord 需要转换一下格式  label用做int64

def int64_feature(value):
#Wrapper for inserting int64 features into Example proto.
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

#image 用作bytes
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


In [ ]:

def convert_to_tfrecord(images, labels, save_dir, name):
    '''

    :param images:  list of iamge directories, string type
    :param labels:  list of labels, int type
    :param save_dir:  the directories to save tfrecord
    :param name:  the name of tfrecord file, string type
    :return:
    '''

    filename = os.path.join(save_dir, name + '.tfrecords')
    number_sample = len(labels)

    if np.shape(images)[0] != number_sample:
        raise ValueError('Images size %d does not match label size %d.'
                         % (images.shape[0], number_sample))

    writer = tf.python_io.TFRecordWriter(filename)
    print('start........')
    for i in np.arange(0, number_sample):
        try:
            image = io.imread(images[i])  # type(image) must be array
            image_raw = image.tostring()  #将图像矩阵转化为一个字符串
            label = int(labels[i])
             #将一个样例转化为Example Protocol Buffer，并将所有需要的信息写入数据结构
            example = tf.train.Example(features=tf.train.Features(feature={
                'label':int64_feature(label),
                'image_raw':bytes_feature(image_raw)
            }))
            #将example写入TFRecord文件
            writer.write(example.SerializeToString())
        except IOError as exc:
            print("maybe con't read the ",images[i])
            print('error: %s' %exc)
            print('skip the image\n')
    writer.close()
    print('done!......')


In [ ]:

def read_and_decode(tfrecord_file, batch_size):  #生成(image,lable)的batch用作测试
    '''

    :param tfrecord_file: the directory of tfrecord file
    :param batch_size:  batch_size hh
    :return: image: 4D tensor - [batch_size, width, height, channel]
             label: 1D tensor - [batch_size]
    '''
    #创建一个producer 为了读入tfrecord
    file_queue = tf.train.string_input_producer([tfrecord_file])
    
    #创建一个reader来读取TFRecord文件中的样例
    #TFRecordReader会一直弹出队列中文件的名字，直到队列为空。
    reader = tf.TFRecordReader()
    _, visual_example = reader.read(file_queue)
    #将字符串解析成图像对应的像素数组
    image_features = tf.parse_single_example(
                                visual_example,
                                features={
                                    'label': tf.FixedLenFeature([], tf.int64),
                                    'image_raw': tf.FixedLenFeature([], tf.string),
                                })
    
    image = tf.decode_raw(image_features['image_raw'], tf.uint8)
    #
    #  
    #
    image = tf.reshape(image, [28, 28])
    label = tf.cast(image_features['label'], tf.int32)
    #使用shuffle_batch可以随机打乱输入
    image_batch, label_batch = tf.train.shuffle_batch([image, label],
                                              batch_size=batch_size,
                                              capacity=2000,
                                              min_after_dequeue=1000
                                              )  #随机获得 batch ，可以调整线程数量，队列中最大元素等
    return image_batch, tf.reshape(label_batch, [batch_size])

'''
比如batch_size=5,capacity=10,min_after_dequeue=5,
初始是有序的0,1，..,9(10条记录)，
然后打乱8,2,6,4,3,7,9,2,0,1(10条记录),
队尾取出5条，剩下7,9,2,0,1(5条记录),
然后又按顺序补充进来，变成7,9,2,0,1,10,11,12,13,14(10条记录)，
再打乱13,10,2,7,0,12...1(10条记录)，
再出队...
'''


In [ ]:

file_dir = '../data/notMNIST_small'
save_dir = '../data'
batch_size = 25
tfrecord_name = 'test'

images,labels = get_file(file_dir)
convert_to_tfrecord(images, labels, save_dir, tfrecord_name)


In [ ]:
def plot_images(images, labels,batch_size):

    for i in np.arange(0,batch_size):
        plt.subplot(np.sqrt(batch_size),np.sqrt(batch_size), i + 1)
        plt.axis('off')
        plt.title(chr(ord('A') + labels[i] - 1),fontsize=14)
        plt.subplots_adjust(top=1.5)
        plt.imshow(images[i])
    plt.show()

In [ ]:
tfrecord_file = '../data/test.tfrecords'
image_batch, label_batch = read_and_decode(tfrecord_file,batch_size)

In [ ]:
with tf.Session() as sess:
    i = 0
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    try:
        while not coord.should_stop() and i < 2:

            image, label = sess.run([image_batch, label_batch])
            plot_images(image, label,batch_size)
            i += 1

    except tf.errors.OutOfRangeError:
        print('done!')
    finally:
        coord.request_stop()
    coord.join(threads)
